In [ ]:
seed = 42
proj_id = 'cmat-315920'
root_path  = '/home/jupyter'

%run -i /home/jupyter/MathVGerrmandering_CMAT_2021/redistrict_code.py
pd.set_option('display.max_columns', None)
states = get_states()

opts = {'census_yr':2010,
        'shapes_yr':2020,
        'election_filters' : {
            "race='general' and office='u.s. sen'",
            "race='general' and office='president'",
            "race='general' and office like 'u.s. rep%'"},
        'overwrite' : {
#             'crosswalks',
#             'assignments',
#             'elections',
#             'census',
#             'shapes',
#             'edges',
#             'graph',
#             'nodes'
        }}

def f(st, lvl, dist):
    opts.update({'abbr':st, 'level':lvl, 'district':dist})
    pprint(opts)
    g = Gerry(**opts)
    g.get_data()
    print('===================================================================================')
    return g


g = f('TX', 'tabblock', 'cd')

# for st in ['TX']:#'RI','TX']:
#     for lvl in ['tract', 'bg', 'tabblock']:
#         for dist in ['cd', 'sldu', 'sldl']:
#             f(st, lvl, dist)

In [ ]:
def MCMC():
    self.nodes['district'] = self.nodes[self.district].copy()
    self.pops = self.nodes.groupby('district')['pop_total'].sum()
    self.pop_ideal = self.pops.mean()
    self.pop_tolerance = max(10, (np.max(self.pops) - np.min(self.pops)) / self.pop_ideal * 100)
    print(f'setting population imbalance tolerance = {self.pop_tolerance}, which is the value for the current map')

    self.districts = self.nodes['district'].unique()
    self.maps = [self.nodes['district'].copy().T]
    for step in range(2):
        recomb_step()
        self.maps.append(self.nodes['district'].copy().T)
    self.maps = pd.concat(self.maps, axis=0)


def recomb_step():
    recom_found = False
    attempts = 0
    for district_pair in rng.permutation([(a,b) for a in self.districts for b in self.districts if a < b]).tolist():
        N = self.nodes.query(f'district in {district_pair}').copy()
        H = self.graph.subgraph(N.index)
        if not nx.is_connected(H):
            print(f'{district_pair} not connected')
            continue
        print(f'{district_pair} connected')
        pops = self.pops.copy()
        p0 = pops.pop(district_pair[0])
        p1 = pops.pop(district_pair[1])
        pop_pair = p0 + p1
        pop_min, pop_max = pops.min(), pops.max()
        trees = []
        for i in range(100):
            w = {e: rng.uniform() for e in H.edges}
            nx.set_edge_attributes(H, w, "weight")
            T = nx.minimum_spanning_tree(H)
            h = hash(tuple(sorted(T.edges)))
            print(h, trees)
            if h not in trees:
                trees.append(h)
                d = {e: T.degree[e[0]] + T.degree[e[1]] for e in T.edges}
                max_tries = 0.02 * len(d)
                print(len(d), max_tries)
                d = sorted(d.items(), key=lambda x:x[1], reverse=True)
                for i, (e, deg) in enumerate(d):
                    if i > max_tries:
                        print(f'I unsuccessfully tried {i} edge cuts for tree {h} - trying a new tree')
                        break
                    if i%100 == 0:
                        print(i, e, deg)
                    attempts += 1
                    T.remove_edge(*e)
                    comp = nx.connected_components(T)
                    next(comp)
                    s = sum(T.nodes[n]['pop_total'] for n in next(comp))
                    t = pop_pair - s
                    if t < s:
                        s, t = t, s
                    pop_imbalance = (max(t, pop_max) - min(s, pop_min)) / self.pop_ideal * 100
#                     print(h, s, t, pop_imbalance)
                    if pop_imbalance < self.pop_tolerance:
                        print(f'found split with pop_imbalance={pop_imbalance}')
                        recom_found = True
                        new = [list(c) for c in nx.connected_components(T)]
#                         N['district'] = 0
                        for n, d in zip(new, district_pair):
                            N.loc[n, 'district_new'] = d
                        i = N.groupby(['district','district_new'])['area'].sum().idxmax()
                        if i[0] != i[1]:
                            new[0], new[1] = new[1], new[0]
                        for n, d in zip(new, district_pair):
                            self.nodes.loc[n, 'district'] = d
                        break
                    T.add_edge(*e)
            else:
                print('Got a repeat spanning tree')
            if recom_found:
                break
        if recom_found:
            break
    assert recom_found, "No suitable recomb step found"
    return recom_found, attempts, trees

rng = np.random.default_rng(42)
self = g
T = MCMC()

In [ ]:
(self.maps.iloc[:,1] != self.maps.iloc[:,2]).sum()

In [ ]:
T

In [ ]:
# for e in T.edges:
#     print(e)
#     print(T.degree[e[0]])
#     break
d = {e:T.degree[e[0]] + T.degree[e[1]] for e in T.edges}
w = sorted(d.keys(), key=d.values())

In [ ]:
# {k:d[k] for k in sorted(d)}
w = sorted(d.items(), key=lambda x:x[1], reverse=True)

for e, deg in w:
    print(e)
    break

In [ ]:
sorted?